In [ ]:
#Insert embeddings into pandas dataframe for inference

In [1]:
import pandas as pd

In [2]:
songEmbed = pd.read_csv('/Users/kushagrakhatwani/Downloads/emb_word2vec_2M.tsv', sep='\t', header = None)

In [5]:
songEmbed.head()

,0,1,2,3,4,5,6,7,8,9,...,22,23,24,25,26,27,28,29,30,31
0,0.170759,-0.169853,0.300353,-0.433129,-0.073195,-0.146205,-0.183122,-0.168517,-0.280988,0.408113,...,0.033185,-0.304589,-0.427648,0.202751,0.138046,-0.393712,0.309896,-0.220839,-0.349681,-0.087443
1,-0.124226,-0.166337,0.335503,-0.362530,-0.149489,-0.403289,-0.004213,0.010922,0.028406,0.143717,...,-0.323346,0.127613,0.193298,0.049566,0.383433,-0.116344,0.546167,0.235206,-0.284042,-0.097694
2,-0.151025,0.491506,0.593315,-0.954169,-0.525090,-0.300619,-0.227297,-0.721840,-0.173667,-0.399060,...,-0.086917,0.086334,0.961037,-0.178883,0.279249,0.136407,0.884335,0.344204,-0.424628,-0.158879
3,-0.040341,-0.368661,0.322449,0.060857,-0.161765,-0.354363,-0.270621,-0.434201,-0.632543,-0.152007,...,0.157554,0.203426,-0.053728,-0.164640,0.497353,-0.001942,0.526925,-0.154109,0.099861,-0.398047
4,-0.580748,-0.323054,-0.463084,0.278989,-0.234253,-0.869133,-0.550567,0.214883,-0.639492,-0.333110,...,0.096742,0.010561,0.125425,0.132857,-0.234901,-0.148464,-0.163571,0.120459,0.068198,0.716602


In [7]:
songEmbed.shape

(2262292, 32)

In [ ]:
#Insert Meta Data into csv for inference

In [9]:
meta = pd.read_csv('/Users/kushagrakhatwani/Downloads/meta_word2vec_2M.tsv', sep='\t', header = None)

In [10]:
meta.head()

,0
0,Encerrado estoy- La 25
1,The Charleston- Pee Wee Hunt
2,Song for Liam- Paris Café Society
3,Save The Last Dance For Me- On-Hold Music
4,Baby Mama- Three 6 Mafia


In [11]:
meta.shape

(2261939, 1)

In [ ]:
# pip install chromadb

In [ ]:
#Use Chroma DB for vector search

In [15]:
import chromadb
from chromadb.config import Settings

In [17]:
# Configure ChromaDB
client = chromadb.Client(Settings())

# Create or connect to a collection
collection = client.create_collection(name="song_embeddings")


In [19]:
# Limit the data to the first 100,000 entries
songEmbed_subset = songEmbed.iloc[:100000]
meta_subset = meta.iloc[:100000]


In [21]:
# Convert song titles and embeddings into lists
song_titles_subset = meta_subset[0].tolist()
embeddings_subset = songEmbed_subset.values.tolist()  # Convert DataFrame to a list of lists
ids = [str(i) for i in range(len(song_titles_subset))] 

In [23]:
batch_size_limit = 40000

# Add data in smaller batches
for start in range(0, len(song_titles_subset), batch_size_limit):
    end = min(start + batch_size_limit, len(song_titles_subset))  # Ensure the last batch is handled properly

    batch_ids = ids[start:end]
    batch_documents = song_titles_subset[start:end]
    batch_embeddings = embeddings_subset[start:end]

    # Add the current batch to the collection
    collection.add(
        ids=batch_ids,
        documents=batch_documents,
        embeddings=batch_embeddings
    )

    print(f"Added batch {start // batch_size_limit + 1} from {start} to {end}")

Added batch 1 from 0 to 40000
Added batch 2 from 40000 to 80000
Added batch 3 from 80000 to 100000


In [ ]:
# Song : Last Christmas- ELIZA (We will get top 5 similar for this song)

In [27]:
import numpy as np
christmasSongEmbedding = songEmbed.iloc[99952].values 

# Check if the embedding is valid
if not np.isnan(christmasSongEmbedding).any():
    # Query the collection with the first song's embedding
    results = collection.query(
        query_embeddings=[christmasSongEmbedding],
        n_results=10  # Adjust this number to control how many similar results you want
    )

    # Display the results
    print("Results for the first song:")
    for i, doc in enumerate(results['documents'][0]):
        print(f"Rank {i+1}: {doc}")
else:
    print("The extracted embedding for the first song is invalid or contains NaN values.")

Results for the first song:
Rank 1: Last Christmas- ELIZA
Rank 2: Holiday Shaving Cream- Joel Samberg & Benny Bell
Rank 3: Christmas Vacation- illScarlett
Rank 4: Christmas Island- The Tesca Country Clan
Rank 5: Go Tell It on the Mountain - Christmas Version- Go tell it on the mountain
Rank 6: Away In A Manger- The Columba Minstrels
Rank 7: Christmas Is A-Comin' (May God Bless You)- Bing Crosby
Rank 8: The First Nowell- Ensamble Clement
Rank 9: Christmas Rebellion- Jars Of Clay
Rank 10: Lo! How a Rose E er Blooming- Southwestern Seminary Oratorio Chorus, The Fortworth Chamber Orchestra


In [ ]:
# Song : White Iverson- Post Malone (We will get top 5 similar for this song)

In [29]:
postMaloneSongembedding = songEmbed.iloc[49132].values 

# Check if the embedding is valid
if not np.isnan(postMaloneSongembedding).any():
    # Query the collection with the first song's embedding
    results = collection.query(
        query_embeddings=[postMaloneSongembedding],
        n_results=5  # Adjust this number to control how many similar results you want
    )

    # Display the results
    print("Results for the first song:")
    for i, doc in enumerate(results['documents'][0]):
        print(f"Rank {i+1}: {doc}")
else:
    print("The extracted embedding for the first song is invalid or contains NaN values.")

Results for the first song:
Rank 1: White Iverson- Post Malone
Rank 2: Broccoli (feat. Lil Yachty)- DRAM
Rank 3: I Serve the Base- Future
Rank 4: Bad Tings- Zoey Dollaz
Rank 5: Home Team- Open Minds
